In [ ]:
import os
import cv2
import dlib
import numpy as np

# Load the predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Paths
trainpath = "database/"
featurespath = "features/"
list_folders = ['1','2','3','4','5','6','7','8','9', '10']  # Modify based on your actual folder names

# Data storage
TrainStruct = []
TestStruct = []

# Histogram equalization function
def Histequal(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.equalizeHist(gray)

# Processing images
for folder in list_folders:
    folder_path = os.path.join(trainpath, folder)
    image_files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith(".bmp")])
    total_images = len(image_files)

    for i, image_file in enumerate(image_files):
        image_path = os.path.join(folder_path, image_file)
        print(f"Processing: {image_path}")
        
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Could not read {image_path}")
            continue
        
        gray = Histequal(img)
        
        faces = detector(gray)
        
        for face in faces:
            landmarks = predictor(gray, face)
            
            key_indices = {
                "left_eyebrow": [18, 19, 20],
                "right_eyebrow": [23, 24, 25],
                "left_eye": [36, 37, 38, 39, 40, 41],
                "right_eye": [42, 43, 44, 45, 46, 47],
                "mouth": [50, 51, 52, 56, 57, 58, 60, 54],
                "nose": [33],
                "head": [27]
            }
            
            points = {idx: (landmarks.part(idx).x, landmarks.part(idx).y) for indices in key_indices.values() for idx in indices}
            
            EARLeftEye = (points[40][1] - points[38][1] + points[41][1] - points[37][1]) / (2 * (points[39][0] - points[36][0]))
            EARRightEye = (points[46][1] - points[44][1] + points[47][1] - points[43][1]) / (2 * (points[45][0] - points[42][0]))
            
            EARLeftEyebrow = (points[27][1] - points[18][1] + points[27][1] - points[19][1] + points[27][1] - points[20][1]) / (points[20][0] - points[18][0])
            EARRightEyebrow = (points[27][1] - points[23][1] + points[27][1] - points[24][1] + points[27][1] - points[25][1]) / (points[25][0] - points[23][0])
            
            EARMOUTH = (points[50][1] - points[58][1] + points[52][1] - points[56][1] + points[51][1] - points[57][1]) / (points[54][0] - points[60][0])
            
            TETA = (points[33][1] - points[27][1]) / (points[25][0] - points[23][0])
            angle = np.arctan(1 / (TETA * 2))
            
            V2 = [EARLeftEye, EARRightEye, EARLeftEyebrow, EARRightEyebrow, EARMOUTH, angle]
            
            if i < total_images / 2:
                TrainStruct.append(V2)
            else:
                TestStruct.append(V2)

# Save features
os.makedirs(featurespath, exist_ok=True)
np.save(os.path.join(featurespath, 'TestStruct.npy'), TestStruct)
np.save(os.path.join(featurespath, 'TrainStruct.npy'), TrainStruct)

print("Feature extraction complete!")
